In [10]:
from keras.models import Model 
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, UpSampling1D
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [11]:
poor = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [16]:
bank_reserves_train, bank_reserves_test = train_test_split(bank_reserves, test_size=0.20, random_state=42)

In [17]:
bank_reserves_train = np.concatenate(bank_reserves_train, axis=1)
bank_reserves_test = np.concatenate(bank_reserves_test, axis=1)

In [18]:
bank_reserves_train.shape

(303, 80000)

In [19]:
input = Input(shape=(303,1))

encoded = Conv1D(filters=10, kernel_size=32, activation='relu', padding='same')(input)
encoded = Conv1D(filters=10, kernel_size=64, activation='relu', padding='same')(encoded)
encoded = Conv1D(filters=10, kernel_size=128, activation='relu', padding='same')(encoded)
encoded = Conv1D(filters=10, kernel_size=128, activation='relu', padding='same')(encoded)

encoded = Dense(384, activation='relu')(encoded)
decoded = Dense(5, activation='relu')(encoded)
decoded = Dense(384, activation='relu')(decoded)

decoded = UpSampling1D(size=2)(decoded)
decoded = Conv1D(filters=10, kernel_size=128, activation='relu')(decoded)
decoded = UpSampling1D(size=2)(decoded)
decoded = Conv1D(filters=10, kernel_size=128, activation='relu')(decoded)
decoded = UpSampling1D(size=2)(decoded)
decoded = Conv1D(filters=10, kernel_size=64, activation='relu')(decoded)
decoded = UpSampling1D(size=2)(decoded)
decoded = Conv1D(filters=10, kernel_size=32, activation='relu')(decoded)

autoencoder = Model(input, decoded)
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 303, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ (None, 303, 10)        │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 303, 10)        │         6,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 303, 10)        │        12,810 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 303, 10)        │        12,810 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 303, 384)       │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 303, 5)         │         1,925 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 303, 384)       │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_5 (UpSampling1D)  │ (None, 606, 384)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 479, 10)        │       491,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_6 (UpSampling1D)  │ (None, 958, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 831, 10)        │        12,810 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_7 (UpSampling1D)  │ (None, 1662, 10)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 1599, 10)       │         6,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_8 (UpSampling1D)  │ (None, 3198, 10)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 3167, 10)       │         3,210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 554,773 (2.12 MB)

 Trainable params: 554,773 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [21]:
bank_reserves_train = bank_reserves_train.astype('float32')
autoencoder.fit(bank_reserves_train, bank_reserves_train,
 epochs=20,
 batch_size=8,
 shuffle=True,
 validation_data=(bank_reserves_test, bank_reserves_test),
 verbose=1)

Epoch 1/20


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 303, 1), found shape=(None, 80000)

In [ ]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=4).output)
encoded_ts = encoder.predict(bank_reserves_test)

In [ ]:
encoded_input = Input(shape=(5,))  # 5 is the size of your latent space
decoded_layer = autoencoder.layers[5](encoded_input)
for layer in autoencoder.layers[6:]:
    decoded_layer = layer(decoded_layer)

decoder = Model(inputs=encoded_input, outputs=decoded_layer)
decoded_ts = decoder.predict(encoded_ts)